Installing packages that will be utilized in RAG

In [ ]:
!pip install langchain langchain-community langchain-core langchain-openai
!pip install chromadb
!pip install tiktoken google-search-results
!pip install pydantic
!pip install typing-inspect typing_extensions
!pip install loguru

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.5/362.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/1

Importing libraries

In [ ]:
import openai
import os

from langchain_community.document_loaders.directory import DirectoryLoader

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.chroma import Chroma

from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from loguru import logger

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

Configure OS environment to utilize OpenAI and LangChain APIs

In [ ]:
os.environ["OPENAI_API_KEY"] = # Insert OpenAI API key here
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = # Insert LangChain API key here

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
directory_path = # Enter path to data folder stored in Google Drive

In [ ]:
from langchain_community.document_loaders import TextLoader

Load data into LangChain using Directory Loader

In [ ]:
loader1 = DirectoryLoader(directory_path, glob = "**/*.txt", show_progress = True, loader_cls = TextLoader)
txt_docs = loader1.load_and_split()

100%|██████████| 23/23 [00:00<00:00, 167.78it/s]


Create embeddings and generate local ChromaDB vector database using the documents provided

In [ ]:
import chromadb
embeddings = OpenAIEmbeddings()
client = chromadb.Client()
# Local vector database containing document embeddings
txt_docsearch = Chroma.from_documents(documents=txt_docs, embedding=embeddings)
#logfile = "output.log"

handler = StreamingStdOutCallbackHandler()

# Define LLM
llm = ChatOpenAI(model_name="gpt-4", temperature=0.6) # "gpt-4"

# Create Retriever
qa_txt = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", callbacks=[handler], retriever=txt_docsearch.as_retriever())

In [ ]:
q = "The MD Program at George Washington (GW) School of Medicine and Health Sciences includes substantial content in Clinical Public Health (population health, health systems science, health policy, and community health) to prepare GW graduates for an expanded scope of practice required to be successful 21st century physicians. Write a 350 character short essay on Ishani Nautiyal's specific experiences related to that aspect of the MD Program? Write in first person tone as if you are Ishani. Cite concrete examples."
qa_txt.invoke(q)

{'query': "The MD Program at George Washington (GW) School of Medicine and Health Sciences includes substantial content in Clinical Public Health (population health, health systems science, health policy, and community health) to prepare GW graduates for an expanded scope of practice required to be successful 21st century physicians. Write a 350 character short essay on Ishani Nautiyal's specific experiences related to that aspect of the MD Program? Write in first person tone as if you are Ishani. Cite concrete examples.",
 'result': 'Through my volunteer work at the Cuyahoga County Juvenile Detention Center, I gained insights into the disparities in underserved communities. This motivated me to advocate for equitable access to healthcare. As the president of Global Medical Brigades, I led initiatives to provide accessible healthcare in rural Honduras, underlining the importance of population health.'}